In [1]:
import pandas as pd
import random

In [2]:
# Veut dire que c’est un passage par adresse des 3 variables IX, IY et IZ.
# 3 variables entières qu’il faut initialiser avant le 1er appel de la fonction.
# Veut dire que la fonction est typée, elle retourne un nombre réel.
def alea(IX, IY, IZ):
    # Déclaration d’une variable inter de type réel
    inter = 0.0
    # A chaque exécution de la fonction, les valeurs des 3 variables sont recalculées, d’où un passage par adresse.
    # % : division entière
    # // : reste de la division entière
    IX = 171 * (IX % 177) - 2 * (IX // 177)
    IY = 172 * (IY % 176) - 35 * (IY // 176)
    IZ = 170 * (IZ % 178) - 63 * (IZ // 178)
    # S’il y a un dépassement de capacité sur une variable, on ajoute un nombre très grand, pour la machine 16 bits,
    # pour ramener à une valeur positive. Ce qui marche sur une machine 16 bits marche sur une 32 ou 64 bits.
    if IX < 0:
        IX = IX + 30269
    if IY < 0:
        IY = IY + 30307
    if IZ < 0:
        IZ = IZ + 30323
    inter = ((IX / 30269) + (IY / 30307) + (IZ / 30323))
    return [round(inter - int(inter), 4),IX, IY, IZ]

In [3]:
IX = 100
IY = 100
IZ = 100

In [4]:
salaire = [(40000, 30000), (30000, 20000), (20000, 15000), (15000, 10000), (10000, 7500), (7500, 5000), (5000, 3000)]
frequenceSalaire = [5, 10, 10, 15, 20, 20, 20]
age = [(61, 53), (52, 41), (40, 31), (30, 21)]
frequenceAge = [30, 30, 20, 20]
ageEmbauche = [(21, 22), (23, 24), (25, 28), (29, 32), (33, 35), (36, 40)]
frequenceAgeEmbauche = [5, 20, 25, 25, 20, 5]
salaire_embauche=[(32000,24000),(24000,16000),(16000,12000),(12000,8000),(8000,6000),(6000,4000),(4000,3000)]
frequenceSalaireEmbauche=[5,5,10,20,20,20,20]
years = [24, 25, 26, 27, 28, 29, 30, 31, 32, 33]

In [5]:
ensemble_data_frames = {}
for j in range(1, 41):
    data_frames = {}
    for i in range(23, 34) :
        data_frames['Employe_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire", "cotisation"])
        data_frames['Retraite_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire"])
        data_frames['newRecru_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire", "cotisation"])
    ensemble_data_frames['Simulation_'+str(j)] = data_frames

In [6]:
data_indices = {}
for j in range(1, 41):
    data_indices['Simulation_'+str(j)] = pd.DataFrame(columns=["year", "TotEmp", "TotRet", "TotCotis", "TotPens", "Reserve", "NouvRet", "NouvRec"])

In [7]:
def cumuleFrequenceIntervale(frequences):
    cumules = []
    indexisation = 0
    for i in frequences:
        cumules.append([indexisation, i+indexisation])
        indexisation += i
    return cumules

In [8]:
def indexFromTable(cumules, aleaNumber):
    i = 0
    for intervale in cumules :
        if intervale[1] > aleaNumber*100 :
            return i
        i+=1
    return i-1

In [9]:
def cotisation(salaire):
    if salaire < 5000:
        return salaire*0.05
    elif salaire < 7000 :
        return salaire*0.06
    elif salaire < 100000 :
        return salaire*0.07
    else:
        return salaire*0.1

In [10]:
def createEmployeRetraite(simulation, year, IX, IY, IZ):
    dfEmploye = ensemble_data_frames["Simulation_"+str(simulation)]["Employe_20"+str(year)]
    dfRetraite = ensemble_data_frames["Simulation_"+str(simulation)]["Retraite_20"+str(year)]
    for index in range(11000):
        if index >=1:
            IX = alea(IX, IY, IZ)[1]
            IY = alea(IX, IY, IZ)[2]
            IZ = alea(IX, IY, IZ)[3]
        salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaire), alea(IX, IY, IZ)[0])
        ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAge), alea(IX, IY, IZ)[0])
        i = index+1
        s = random.randrange(salaire[salaireIndex][1], salaire[salaireIndex][0], 100)
        if index <10000:
            a = random.randrange(age[ageIndex][1], age[ageIndex][0], 1)

            new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s], "cotisation" : [cotisation(s)]})
            dfEmploye = pd.concat([dfEmploye, new_row], ignore_index=True)
        else:
            a = random.randint(65, 85)
            new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
            dfRetraite = pd.concat([dfRetraite, new_row], ignore_index=True)
    ensemble_data_frames["Simulation_"+str(simulation)]["Employe_20"+str(year)] = dfEmploye
    ensemble_data_frames["Simulation_"+str(simulation)]["Retraite_20"+str(year)] = dfRetraite

In [11]:
def pensionnaireFive(dfRetrait) :
    pensionnaire65 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    for index, row in dfRetrait.iterrows():
        a = row["age"]
        s = row["salaire"]
        p = ((25*2)/100)*row["salaire"]
        new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
        pensionnaire65 = pd.concat([pensionnaire65, new_row], ignore_index=True)
    return pensionnaire65

In [12]:
def pensionnaireTwo(dfEmploye, dfRetraite):
    pensionnaire62 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    pensionnaire65 = pensionnaireFive(dfRetraite)
    pensionnaire62 = pd.concat([pensionnaire62, pensionnaire65], ignore_index=True)
    for index, row in dfEmploye.iterrows():
        if row["age"]>=62:
            a = row["age"]
            s = row["salaire"]
            p = ((25*2)/100)*row["salaire"]
            new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
            pensionnaire62 = pd.concat([pensionnaire62, new_row], ignore_index=True)
    return pensionnaire62

In [13]:
def newRecrutement(IX, IY, IZ):
    Nouveux_Recrus = pd.DataFrame(columns=["id", "age", "salaire"])
    a=alea(IX, IY, IZ)[0]
    n = ((400 - 250) * (random.uniform(0, 1)*a)) + 250
    for index in range(int(n)):
        if index >= 0 :
            IX = alea(IX, IY, IZ)[1]
            IY = alea(IX, IY, IZ)[2]
            IZ = alea(IX, IY, IZ)[3]
        salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaireEmbauche), alea(IX, IY, IZ)[0])
        ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAgeEmbauche), alea(IX, IY, IZ)[0])
        i = index+1
        s = random.randrange(salaire_embauche[salaireIndex][1], salaire_embauche[salaireIndex][0], 100)
        a = random.randrange(ageEmbauche[ageIndex][0], ageEmbauche[ageIndex][1], 1)
        new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s], "cotisation" : [cotisation(s)]})
        Nouveux_Recrus = pd.concat([Nouveux_Recrus, new_row], ignore_index=True)
    return Nouveux_Recrus

In [14]:
def calculEmployeRetraite(simulation, i, IX, IY, IZ):
    EmployeBeta = ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i-1)]
    RetraiteBeta = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i-1)]
    EmployeAlpha = ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i)]
    RetraiteAlpha = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i)]
    for index, row in EmployeBeta.iterrows():
        if row["age"] < 64:
            if i == 25 or i == 30:
                s = row["salaire"]*1.05
            else :
                s = row["salaire"]
            a = row["age"]+1
            new_row = pd.DataFrame({"id" : row["id"], "age": [a], "salaire": [s], "cotisation" : [cotisation(s)]})
            EmployeAlpha = pd.concat([EmployeAlpha, new_row], ignore_index=True)
        else :
            if i == 25 or i == 30:
                s = row["salaire"]*1.05
            else :
                s = row["salaire"]
            a = row["age"]+1
            new_row = pd.DataFrame({"id" : row["id"], "age": [a], "salaire": [s]})
            RetraiteAlpha = pd.concat([RetraiteAlpha, new_row], ignore_index=True)
    for index, row in RetraiteBeta.iterrows():
        new_row = pd.DataFrame({"id" : [row["id"]], "age": [row["age"]+1], "salaire": [row["salaire"]]})
        RetraiteAlpha = pd.concat([RetraiteAlpha, new_row], ignore_index=True)
    newRecru = newRecrutement(IX, IY, IZ)
    for index, row in newRecru.iterrows():
        new_row = pd.DataFrame({"id" : [row["id"]], "age": [row["age"]], "salaire": [row["salaire"]],"cotisation" : [row["cotisation"]]})
        EmployeAlpha = pd.concat([EmployeAlpha, new_row], ignore_index=True)
    ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i)] = EmployeAlpha
    ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i)] = RetraiteAlpha
    ensemble_data_frames["Simulation_"+str(simulation)]['newRecru_20'+str(i)] = newRecru

In [15]:
def calculIndice(simulation, ageRetrait):
    dfIndice = data_indices["Simulation_"+str(simulation)]
    dfEmploye =  ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(23)]
    dfRetraite = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(23)]
    TotEmp = dfEmploye.shape[0]
    if ageRetrait == 62:
        TotCotis = dfEmploye[dfEmploye["age"]<=61]["cotisation"].sum()*12
        pensionnaire = pensionnaireTwo(dfEmploye, dfRetraite)
    else:
         TotCotis = dfEmploye["cotisation"].sum()*12
         pensionnaire = pensionnaireFive(dfRetraite)
    TotRet = pensionnaire.shape[0]
    TotPens = pensionnaire["pension"].sum()*12
    Reserve = 200000000 + TotCotis - TotPens
    NouvRet = 0
    NouvRec = 0
    new_row = pd.DataFrame({"year" : [2023], "TotEmp" : [TotEmp], "TotRet" : [TotRet], "TotCotis" : [TotCotis], "TotPens" : [TotPens], "Reserve" : [Reserve], "NouvRet" : [NouvRet], "NouvRec" : [NouvRec]})
    dfIndice = pd.concat([dfIndice, new_row], ignore_index=True)
    for year in years :
        dfEmploye =  ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(year)]
        dfRetraite = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)]
        dfRecrutement = ensemble_data_frames["Simulation_"+str(simulation)]['newRecru_20'+str(year)]
        TotEmp = dfEmploye.shape[0]
        if ageRetrait == 62:
            TotCotis = dfEmploye[dfEmploye["age"]<=61]["cotisation"].sum()*12
            pensionnaire = pensionnaireTwo(dfEmploye, dfRetraite)
        else:
            pensionnaire = pensionnaireFive(dfRetraite)
            TotCotis = dfEmploye["cotisation"].sum()*12
        TotPens = pensionnaire["pension"].sum()*12
        TotRet = pensionnaire.shape[0]
        Reserve = dfIndice.iloc[year-24]['Reserve'] + TotCotis -TotPens
        NouvRet = TotRet - dfIndice.iloc[year-24]['TotRet']
        NouvRec =  dfRecrutement.shape[0]
        new_row = pd.DataFrame({"year" : [2000+year], "TotEmp" : [TotEmp], "TotRet" : [TotRet], "TotCotis" : [TotCotis], "TotPens" : [TotPens], "Reserve" : [Reserve], "NouvRet" : [NouvRet], "NouvRec" : [NouvRec]})
        dfIndice = pd.concat([dfIndice, new_row], ignore_index=True)
    return dfIndice

In [16]:
createEmployeRetraite(1, 23, IX, IY, IZ)
for year in years:
    calculEmployeRetraite(1, year, IX, IY, IZ)

In [17]:
simulation = 1
year = 33
ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(year)]

,id,age,salaire,cotisation
0,1,41,6835.50,410.1300
1,2,52,10694.25,748.5975
2,4,38,3528.00,176.4000
3,5,31,4299.75,214.9875
4,6,31,4740.75,237.0375
...,...,...,...,...
10809,330,27,9200.00,644.0000
10810,331,23,12200.00,854.0000
10811,332,26,6800.00,408.0000
10812,333,26,8300.00,581.0000


In [18]:
ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)]

,id,age,salaire
0,3,65,29326.50
1,48,65,11907.00
2,64,65,13230.00
3,86,65,42005.25
4,95,65,13119.75
...,...,...,...
3223,10996,82,9600.00
3224,10997,89,37000.00
3225,10998,82,3700.00
3226,10999,91,7800.00


In [19]:
pensionnaireTwo(ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(year)], ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)])

,id,age,salaire,pension
0,3,65,29326.50,14663.250
1,48,65,11907.00,5953.500
2,64,65,13230.00,6615.000
3,86,65,42005.25,21002.625
4,95,65,13119.75,6559.875
...,...,...,...,...
3979,9885,63,19955.25,9977.625
3980,9918,64,21168.00,10584.000
3981,9949,64,11466.00,5733.000
3982,9987,64,13781.25,6890.625


In [20]:
pensionnaireFive(ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)])

,id,age,salaire,pension
0,3,65,29326.50,14663.250
1,48,65,11907.00,5953.500
2,64,65,13230.00,6615.000
3,86,65,42005.25,21002.625
4,95,65,13119.75,6559.875
...,...,...,...,...
3223,10996,82,9600.00,4800.000
3224,10997,89,37000.00,18500.000
3225,10998,82,3700.00,1850.000
3226,10999,91,7800.00,3900.000


In [21]:
ensemble_data_frames["Simulation_"+str(simulation)]['newRecru_20'+str(year)]

,id,age,salaire,cotisation
0,1,31,6000,360.0
1,2,23,14200,994.0
2,3,36,3400,170.0
3,4,33,3900,195.0
4,5,34,3300,165.0
...,...,...,...,...
329,330,27,9200,644.0
330,331,23,12200,854.0
331,332,26,6800,408.0
332,333,26,8300,581.0


In [22]:
calculIndice(1, 62)

,year,TotEmp,TotRet,TotCotis,TotPens,Reserve,NouvRet,NouvRec
0,2023,10000,1000,94609572.00,67734600.0,2.268750e+08,0,0
1,2024,10345,1000,96953208.00,67734600.0,2.560936e+08,0,345
2,2025,10672,1352,97379748.00,117471840.0,2.360015e+08,352,327
3,2026,10935,1731,91711569.00,170766690.0,1.569464e+08,379,263
4,2027,11253,2097,86897206.20,220520310.0,2.332326e+07,366,318
5,2028,11173,2470,81350661.60,273165000.0,-1.684911e+08,373,272
6,2029,11078,2857,75591634.80,327757020.0,-4.206565e+08,387,284
7,2030,11026,3228,74717632.65,387289531.5,-7.332284e+08,371,314
8,2031,10988,3613,69008892.69,444599245.5,-1.108819e+09,385,335
9,2032,10851,3984,63188355.75,498348766.5,-1.543979e+09,371,250


In [23]:
calculIndice(1, 65)

,year,TotEmp,TotRet,TotCotis,TotPens,Reserve,NouvRet,NouvRec
0,2023,10000,1000,9.460957e+07,67734600.0,2.268750e+08,0,0
1,2024,10345,1000,9.695321e+07,67734600.0,2.560936e+08,0,345
2,2025,10672,1000,1.043430e+08,67734600.0,2.927019e+08,0,327
3,2026,10935,1000,1.061361e+08,67734600.0,3.311034e+08,0,263
4,2027,11253,1000,1.082872e+08,67734600.0,3.716560e+08,0,318
5,2028,11173,1352,1.031477e+08,117471840.0,3.573319e+08,352,272
6,2029,11078,1731,9.757028e+07,170766690.0,2.841355e+08,379,284
7,2030,11026,2097,9.771705e+07,223007991.0,1.588445e+08,366,314
8,2031,10988,2470,9.229290e+07,278284915.5,-2.714750e+07,373,335
9,2032,10851,2857,8.597227e+07,335606536.5,-2.767818e+08,387,250


In [24]:
ListSimulation = []
for i in range (40):
    if i == 0 :
        ListSimulation.append({"IX": 100, "IY" : 100, "IZ": 100})
    else :
        ListSimulation.append({"IX": ListSimulation[i-1]["IX"] + 10, "IY": ListSimulation[i-1]["IY"] + 20, "IZ": ListSimulation[i-1]["IZ"] + 30})

In [25]:
def createSimulation(Simulation, X, Y, Z) :
    createEmployeRetraite(Simulation, 23, X, Y, Z)
    for year in years:
        calculEmployeRetraite(Simulation, year, X, Y, Z)

In [26]:
createSimulation(2, ListSimulation[1]["IX"], ListSimulation[1]["IY"], ListSimulation[1]["IZ"])

In [ ]:
for i in range(3, 41):
    createSimulation(i, ListSimulation[i-1]["IX"], ListSimulation[i-1]["IY"], ListSimulation[i-1]["IZ"])

In [ ]:
ensemble_data_frames["Simulation_22"]['Retraite_2032']

In [ ]:
calculIndice(21, 62)

In [ ]:
calculIndice(21, 65)